In [2]:
import matplotlib.pyplot as plt
import matplotlib.collections
import numpy as np
from cartpole import CartPole

In [3]:
env = CartPole(visual=False)
env.sim_steps = 50         # number of Euler integration steps to perform in one go
env.delta_time = 0.05

### Fit a Linear Model Using Least Squares

In [124]:
n_samples = 500

X = []
Y = []

for _ in range(n_samples):
    cart_pos = np.random.uniform(-5, 5)
    cart_vel = np.random.uniform(-10, 10)
    pole_ang = np.random.uniform(-np.pi, np.pi)
    pole_vel = np.random.uniform(-15, 15)

    init_state = np.array([cart_pos, cart_vel, pole_ang, pole_vel])

    env.reset()
    env.setState(init_state.copy())
    env.performAction(action=0.0)
    next_state = env.getState().copy()

    X.append(init_state.copy())
    Y.append(next_state - init_state)

X = np.array(X)
Y = np.array(Y)

np.save('X_data.npy', X)
np.save('Y_data.npy', Y)


# Solve for linear model C such that Y ≈ X @ C.T
C_T, _, _, _ = np.linalg.lstsq(X, Y, rcond=None)
C = C_T.T

np.save('linear_C.npy', C)

print(C)

[[-1.27939967e-03  1.00506972e-01  3.52469170e-02  1.29984458e-04]
 [-1.27939967e-02  5.06972425e-03  3.52469170e-01  1.29984458e-03]
 [-2.14671896e-03 -6.98534942e-04  1.29490103e-01  9.93636238e-02]
 [-2.14671896e-02 -6.98534942e-03  1.29490103e+00 -6.36376229e-03]]


### Compare Prediction Against Training Data

In [ ]:
Y_pred = X @ C.T
labels = ['Cart Position', 'Cart Velocity', 'Pole Angle', 'Pole Angular Velocity']
plt.figure(figsize=(12, 8))

for i in range(4):
    true_vals = Y[:, i]
    pred_vals = Y_pred[:, i]
    error = -1*(np.abs(pred_vals - true_vals)**2)

    plt.subplot(2, 2, i+1)
    sc = plt.scatter(true_vals, pred_vals, c=error, s=5, cmap='viridis', alpha=0.5)
    
    min_val = min(true_vals.min(), pred_vals.min())
    max_val = max(true_vals.max(), pred_vals.max())
    # y = x line
    plt.plot([min_val, max_val], [min_val, max_val], color='black', linestyle='--', linewidth=1)

    plt.xlabel('True ' + labels[i])
    plt.ylabel('Predicted ' + labels[i])
    plt.title(labels[i])
    plt.grid(True)
    plt.colorbar(sc, label='Negative of Prediction Error Squared')

# plt.savefig('1.3_predict_vs_true.png')
plt.tight_layout()
plt.show()

### 1D Scan

In [33]:
# compare with scans
true_deltas = [[] for _ in range(4)]
pred_deltas = [[] for _ in range(4)]

# deterministic
cart_pos = 0
cart_vel = -7.644
pole_ang = 2.744
pole_vel = 7.176

for pole_ang in pole_ang_scan:
    init_state = np.array([cart_pos, cart_vel, pole_ang, pole_vel])

    # True delta from environment
    env.setState(init_state)
    env.performAction(action=0.0)
    next_state = env.getState().copy()
    delta_true = next_state - init_state
    delta_pred = init_state @ C.T

    for i in range(4):
        true_deltas[i].append(delta_true[i])
        pred_deltas[i].append(delta_pred[i])

In [ ]:
plt.figure(figsize=(10, 8))
for i in range(4):
    plt.subplot(2, 2, i + 1)
    plt.plot(pole_ang_scan, true_deltas[i], label='True Δ', color='blue')
    plt.plot(pole_ang_scan, pred_deltas[i], label='Predicted Δ', color='red', linestyle='--')
    plt.xlabel('Initial Pole Angle')
    plt.ylabel(f'Δ {labels[i]}')
    plt.title(f'Varying Pole Angle')
    plt.grid(True)
    plt.legend()

plt.tight_layout()
# plt.savefig('1.3_pole_angle.png')
plt.show()

### Prediction of Rollouts

In [41]:
env.delta_time = 0.1

In [42]:
# large initial velocities
init_state = np.array([0.0, 7, np.pi, 17])
env.setState(init_state)

num_steps = 50
states = []
states.append(init_state.copy())
pred_states = []
pred_states.append(init_state.copy())

for _ in range(num_steps):
    env.performAction(action=0.0)
    env.remap_angle()
    states.append(env.getState().copy())
    
    next_pred = pred_states[-1] @ C.T + pred_states[-1]
    next_pred[2] = CartPole._remap_angle(next_pred[2])
    pred_states.append(next_pred.copy())

states = np.array(states)
pred_states = np.array(pred_states)

In [ ]:
# plot the state trajectory
time = [i * env.delta_time for i in range(num_steps+1)]
labels = ['Cart Position', 'Cart Velocity', 'Pole Angle', 'Pole Angular Velocity']
title = "Small Oscillation about Stable Equilibirum"


for i in range(4):
    plt.figure(figsize=(6, 4))
    plt.plot(time, states[:, i], label='True Evolution', color='blue')
    plt.plot(time, pred_states[:, i], label='Predicted Evolution', color='red', linestyle='--')
    plt.title(title)
    plt.ylabel(labels[i])
    plt.xlabel('Time (seconds)')
    plt.grid(True)
    # plt.savefig(f"1.4_smallosc_{labels[i].replace(' ', '_').lower()}_vs_time.png")

plt.figure(figsize=(6, 4))
plt.plot(states[:, 1], states[:, 3], label='True Evolution', color='blue')
plt.xlabel('Cart Velocity')
plt.ylabel('Pole Angular Velocity')
plt.title(title + ' - True Evolution')
plt.grid(True)
# plt.savefig("1.4_smallosc_true.png")

plt.figure(figsize=(6, 4))
plt.plot(pred_states[:, 1], pred_states[:, 3], label='Predicted Evolution', color='red', linestyle='--')
plt.xlabel('Cart Velocity')
plt.ylabel('Pole Angular Velocity')
plt.title(title + ' - Predicted Evolution')
plt.grid(True)
# plt.savefig("1.4_smallosc_pred.png")

plt.tight_layout()
plt.show()

In [ ]:
def plot_colored_phase(ax, x, y, label):
    """
    Plot a colored phase trajectory where color encodes progression
    through the state sequence.
    """
    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)

    idx_norm = np.array([i * env.delta_time for i in range(len(x))])
    lc = matplotlib.collections.LineCollection(segments, array=idx_norm, cmap='viridis', linewidth=0.5)
    ax.add_collection(lc)

    ax.set_xlim(np.min(x), np.max(x))
    ax.set_ylim(np.min(y), np.max(y))
    ax.set_xlabel('Cart Velocity')
    ax.set_ylabel('Pole Angular Velocity')
    ax.set_title(label)
    ax.grid(True)
    return lc

# Plot for True Evolution
fig1, ax1 = plt.subplots(figsize=(6, 4))
lc1 = plot_colored_phase(ax1, states[:, 1], states[:, 3], "True Phase Trajectory")
plt.colorbar(lc1, ax=ax1, label='Time (Seconds)')
# plt.savefig('1.4_large_contour_true.png')

# Plot for Predicted Evolution
fig2, ax2 = plt.subplots(figsize=(6, 4))
lc2 = plot_colored_phase(ax2, pred_states[:, 1], pred_states[:, 3], "Predicted Phase Trajectory")
plt.colorbar(lc2, ax=ax2, label='Time (Seconds)')
# plt.savefig('1.4_large_contour_pred.png')
plt.show()